In [1]:
import sys
sys.path.insert(0, "./../")

In [2]:
from app.risk_evaluator import RiskEvaluator

In [3]:
from tqdm import _tqdm_pandas

In [4]:
from openai import OpenAI

In [5]:
from app.processor import PDFToSentenceEmbedding
from app.mongodb_engine import MongoDB
from app import utils

In [6]:
embedding_generator = PDFToSentenceEmbedding()
mongo_db_engine = MongoDB()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

NameError: name 'os' is not defined

In [ ]:
file_name = "SUNY RF - General Investment Policy and Guidelines.pdf"

In [ ]:
def get_answer(context, question):
    query_vector = embedding_generator.model.encode(
        context).tolist()

    retrieved_results = mongo_db_engine.hybrid_search(
        query_vector=query_vector, query=context,
        file_name=file_name, limit=5)

    prompt = utils.generate_prompt(retrieved_results)

    completion = client.chat.completions.create(
        model='gpt-3.5-turbo-16k',
        messages=[
            {'role': 'system',
                'content': prompt,
             },
            {"role": "user",
                "content": question},
            {"role": "user",
                "content":
                """The answer should follow below json format: 
                {"fullfil":"{YES OR NO}",
                "page_number":"{page_number}",
                "reason":"{reason}"}}"""
             }
        ],
        temperature=0,
        stream=False
    )

    answer = completion.choices[0].message.content

    answer_dict = json.loads(answer)

    answer_dict['criteria'] = question

    return answer_dict

In [ ]:
def get_risk_level(criteria_fulfillment_list):
    num_fulfil = 0
    for fulfillment in criteria_fulfillment_list:
        if fulfillment['answer'].upper() == 'YES':
            num_fulfil += 1

    if num_fulfil >= 4:
        return "HIGH RISK"

    if num_fulfil >= 3:
        return "MEDIUM RISK"

    return "LOW RISK"

In [ ]:
criteria_list = [
    {"context": "allocation of bonds",
     "question": "Is the percentage of bonds larger than 20% ?"
     },
    {"context": "percentage of hedged equities",
     "question": "Is the percentage of hedged equities larger than 30% ?"
     },
    {"context": "asset allocation compliance policy",
     "question": "Is there any asset allocation compliance policy ?"
     },
    {"context": "allocation of asset with more than 10 year period",
     "question": "allocation of asset with more than 10 year period is larger than 20% ?"
     },
]

In [ ]:
criteria_fulfillment_list = []

In [ ]:
for criteria in tqdm(criteria_list):
    answer_dict = get_answer(**criteria)
    criteria_fulfillment_list.append(answer_dict)

100%|██████████| 4/4 [00:53<00:00, 13.40s/it]


In [ ]:
criteria_fulfillment_list

[{'fulfill': 'YES',
  'page_number': '7',
  'reason': 'The percentage of bonds is larger than 20% according to the asset allocation policy on page 7.',
  'criteria': 'Is the percentage of bonds larger than 20% ?'},
 {'fulfill': 'NO',
  'page_number': '11',
  'reason': 'The percentage of hedged equities is not larger than 30%. According to the Investment Policy and Guidelines on page 11, the long-term target range for hedged equities is 0-25%.',
  'criteria': 'Is the percentage of hedged equities larger than 30% ?'},
 {'fulfill': 'YES',
  'page_number': '6',
  'reason': 'The investment guidelines and asset allocation section on page 6 mentions that if any asset class weighting is outside its respective permissible range by +/- 5 percent at the end of any calendar quarter, the investment consultant will advise the Committee at the next Committee meeting. The portfolio should be re-balanced to bring the portfolio into compliance with these guidelines as promptly and prudently as possible.

In [ ]:
risk_definition = """"
if fulfill all criteria, return HIGH RISK
if fulfill criteria more than 2 or equal to 3, return MEDIUM RISK
if fulfill criteria less than 2 criteria return LOW RISK
"""

In [ ]:
criteria_fulfillment_list_json = json.dumps(criteria_fulfillment_list)

In [ ]:
criteria_fulfillment_list_json

'[{"fulfill": "YES", "page_number": "7", "reason": "The percentage of bonds is larger than 20% according to the asset allocation policy on page 7.", "criteria": "Is the percentage of bonds larger than 20% ?"}, {"fulfill": "NO", "page_number": "11", "reason": "The percentage of hedged equities is not larger than 30%. According to the Investment Policy and Guidelines on page 11, the long-term target range for hedged equities is 0-25%.", "criteria": "Is the percentage of hedged equities larger than 30% ?"}, {"fulfill": "YES", "page_number": "6", "reason": "The investment guidelines and asset allocation section on page 6 mentions that if any asset class weighting is outside its respective permissible range by +/- 5 percent at the end of any calendar quarter, the investment consultant will advise the Committee at the next Committee meeting. The portfolio should be re-balanced to bring the portfolio into compliance with these guidelines as promptly and prudently as possible.", "criteria": "I

In [ ]:
completion = client.chat.completions.create(
    model='gpt-3.5-turbo-16k',
    messages=[
        {"role": "user",
            "content":
            f"""
RISK DEFINITION: 
{risk_definition}

FULFILLED CRITERIA IN JSON Format:
{criteria_fulfillment_list_json}
""" + """please return risk level by following this json format {"risk_level":"{risk_level}"}"}"""},
    ],
    temperature=0,
    stream=False
)

answer = completion.choices[0].message.content

In [ ]:
json.loads(answer)

{'risk_level': 'MEDIUM RISK'}